In [1]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [2]:
# url para o cojunto de dados 
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1839/periodos/2022/variaveis/630?localidades=N1[all]"

In [3]:
spark = SparkSession.builder.getOrCreate()

/home/airflow/.local/lib/python3.11/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
response = requests.get(url=url)
response.raise_for_status()

In [5]:
data = response.json()

In [6]:
schema = T.StructType([
    T.StructField("id", T.StringType(), True),
    T.StructField("variavel", T.StringType(), True),
    T.StructField("unidade", T.StringType(), True),
    T.StructField("resultados", T.ArrayType(T.StructType([
        T.StructField("classificacoes", T.ArrayType(T.StructType([
            T.StructField("id", T.StringType(), True),
            T.StructField("nome", T.StringType(), True),
            T.StructField("categoria", T.MapType(T.StringType(), T.StringType()), True)
        ])), True),
        T.StructField("series", T.ArrayType(T.StructType([
            T.StructField("localidade", T.StructType([
                T.StructField("id", T.StringType(), True),
                T.StructField("nivel", T.StructType([
                    T.StructField("id", T.StringType(), True),
                    T.StructField("nome", T.StringType(), True)
                ]), True),
                T.StructField("nome", T.StringType(), True)
            ]), True),
            T.StructField("serie", T.MapType(T.StringType(), T.StringType()), True)
        ])), True)
    ])), True)
])

In [7]:
df = spark.createDataFrame(data, schema)

In [8]:
# df_out = df.select(
#     F.col("id"),
#     F.col("variavel"),
#     F.col("unidade"),
#     F.col("resultados")[0].getItem("classificacoes")[0].getItem("id").alias("classificacoes_id"),
#     F.col("resultados")[0].getItem("classificacoes")[0].getItem("nome").alias("classificacoes_nome"),
#     F.col("resultados")[0].getItem("classificacoes")[0].getItem("categoria").alias("classificacoes_categoria"),
#     F.col("resultados")[0].getItem("series")[0].getItem("localidade").getItem("id").alias("series_id"),
#     F.col("resultados")[0].getItem("series")[0].getItem("localidade").getItem("nome").alias("series_nome"),
#     F.col("resultados")[0].getItem("series")[0].getItem("localidade").getItem("nivel").getItem("id").alias("series_nivel_id"),
#     F.col("resultados")[0].getItem("series")[0].getItem("localidade").getItem("nivel").getItem("nome").alias("series_nivel_nome"),
#     F.col("resultados")[0].getItem("series")[0].getItem("serie").alias("series")
# )

In [9]:
# df_out = df_out.select("*", F.explode("classificacoes_categoria").alias("categoria_id", "categoria_nome"))
# df_out = df_out.select("*", F.explode("series").alias("series_id", "series_nome"))

# df_out = df_out.drop("classificacoes_categoria", "series")

In [10]:
# df_out.show()

In [11]:
df_exploded_resultados = df.withColumn("resultado", F.explode("resultados"))

In [12]:
df_exploded_classificacoes = df_exploded_resultados.withColumn("classificacao", F.explode("resultado.classificacoes"))

In [13]:
df_exploded_series = df_exploded_classificacoes.withColumn("serie", F.explode("resultado.series"))

In [14]:
df_selected = df_exploded_series.select(
    F.col("id").alias("id"),
    F.col("variavel").alias("variavel"),
    F.col("unidade").alias("unidade"),
    F.col("classificacao.id").alias("classificacao_id"),
    F.col("classificacao.nome").alias("classificacao_nome"),
    F.col("classificacao.categoria").alias("classificacao_categoria"),
    F.col("serie.localidade.id").alias("localidade_id"),
    F.col("serie.localidade.nivel.id").alias("localidade_nivel_id"),
    F.col("serie.localidade.nivel.nome").alias("localidade_nivel_nome"),
    F.col("serie.localidade.nome").alias("localidade_nome"),
    F.col("serie.serie").alias("serie")
)

In [15]:
df_selected.show(truncate=False)

+---+------------------+--------+----------------+-------------------------+-----------------------+-------------+-------------------+---------------------+---------------+----------------+
|id |variavel          |unidade |classificacao_id|classificacao_nome       |classificacao_categoria|localidade_id|localidade_nivel_id|localidade_nivel_nome|localidade_nome|serie           |
+---+------------------+--------+----------------+-------------------------+-----------------------+-------------+-------------------+---------------------+---------------+----------------+
|630|Número de empresas|Unidades|319             |Faixas de pessoal ocupado|{104029 -> Total}      |1            |N1                 |Brasil               |Brasil         |{2022 -> 346105}|
|630|Número de empresas|Unidades|9117            |Tipo de indústria        |{99735 -> Total}       |1            |N1                 |Brasil               |Brasil         |{2022 -> 346105}|
+---+------------------+--------+----------------+